In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap

cmap = get_cmap('bwr')

plt.style.use('seaborn-dark')

plt.rcParams['axes.grid'] = True
plt.rcParams['figure.figsize'] = (15, 5)

from scipy import signal
from itertools import product

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import matplotlib.ticker as ticker



In [2]:
from theory.core import TheoreticalWavelet, Pipe, Rock

## Set local relative directory name

In [ ]:
os.chdir('/data/Orica_project/measurements/LOGPROC/swc/modeling-multipass/20-100Hz/Multiple_convolved_rock-bit')

# This is a function where our feature extraction is happening

In [ ]:
def feature_extractor(wavelet, time, 
                      left_window=None, 
                      center_window=None, 
                      right_window=None,
                      left_pick='min',
                      center_pick='max',
                      right_pick='min',
                      upsample_to=100000):
    time = np.linspace(time.min(), time.max(), upsample_to)
    wavelet = signal.resample(wavelet, upsample_to)

    extracted_features = {}
    for pick, window, window_name in  zip([center_pick, left_pick, right_pick], [center_window, left_window, right_window], ['center', 'left', 'right']):
        if window:
            constrained_indices = np.where((time > window[0]) & (time < window[1]))

            constrained_wavelet = np.zeros_like(wavelet)
            constrained_wavelet[constrained_indices] = wavelet[constrained_indices]
            if pick == 'max':
                feature_index = np.argmax(constrained_wavelet)
            elif pick == 'min':
                feature_index = np.argmin(constrained_wavelet)
            else:
                raise ValueError('picks must be "min" or "max"')

            delay = time[feature_index]
            amplitude = wavelet[feature_index]
            extracted_features[window_name] = {
                'delay': delay,
                'amplitude': amplitude,
                'feature_index': feature_index,
            }
    try:
        extracted_features['center_to_left_ratio'] = extracted_features['center']['amplitude'] / extracted_features['left']['amplitude']
        extracted_features['right_to_left_ratio'] = extracted_features['right']['amplitude'] / extracted_features['left']['amplitude']
    except:
        pass
    return extracted_features

# PLAYGROUND BEFORE MODELING 

This is where we can visualize the wavelets with the pick on top of it given a range of values.
Its useful to visualize what times to expect and plan the windows boundaries.

# CRITICAL --   These are the user selected values carried through the Notebook.

In [ ]:
COMPONENT = 'axial'
ALPHA_OR_BETA_RANGE = np.arange(500, 2250 + 50, 50)
#ALPHA_OR_BETA_RANGE = np.arange(1000, 4000 + 50, 40)

#COMPONENT = 'tangential'
#ALPHA_OR_BETA_RANGE = np.arange(300, 2000 + 50, 25)

WINDOW = 510 # window in time domain by number of samples

# RHO_RANGE = np.arange(1500, 3000 + 500, 500)
RHO_RANGE = np.asarray([2000])

# BIT RADIUS
Rb = 0.14 # radius of the bit contacting rock

BANDPASS_FILTER = [10 , 20, 100, 120]
#BANDPASS_FILTER = [30 , 45, 160, 200]

#  Set the value for number of pipes in multipass.   You need a copy of the Notebook for each value of NUM_PIPES.
#NUM_PIPES = 1
NUM_PIPES = 6

RC_AXIAL_BITSUB = 0.293 # reflection coefficient
DELAY_AXIAL_BITSUB = 0.421 # in ms
RC_AXIAL_Connector = 0.086 # reflection coefficient  (equals RC bitsub squared)
DELAY_AXIAL_Connector = 0.250 # in ms

RC_AXIAL_TOPSUB = -0.55
DELAY_AXIAL_PIPE = 5.627 # in ms

RC_AXIAL_M1A = -0.293 # reflection coefficient
RC_AXIAL_M1B = 0.707 # includes transmission coefficient through pipe-bitsub ; using RC parameter to account for TC scalar effect
RC_AXIAL_M1C = 0.207 # includes RC=0.293 of bitsub-pipe interface mutplied by TC=0.707 for wavelet coming down through pipe-bitsub
DELAY_AXIAL_M1A = 6.095 # in ms     two-way in pipe and topsub or bullsub, etc
DELAY_AXIAL_M1B = 6.516 # in ms     two-way in pipe and topsub plus two-way in bitsub
DELAY_AXIAL_M1C = 6.937 # in ms     two-way in pipe and topsub plus twice two-way in bitsub


RC_TANGENTIAL_BITSUB = -0.120  # reflection coefficient
DELAY_TANGENTIAL_BITSUB = 0.867 # in ms
RC_TANGENTIAL_Connector = 0.0144  # reflection coefficient
DELAY_TANGENTIAL_Connector = 0.515 # in ms

RC_TANGENTIAL_TOPSUB = 0.55
DELAY_TANGENTIAL_PIPE = 11.584  # in ms

RC_TANGENTIAL_M1A = 0.120 # reflection coefficient
RC_TANGENTIAL_M1B = 0.880 # includes transmission coefficient through pipe-bitsub ; using RC parameter to account for TC scalar effect
RC_TANGENTIAL_M1C = 0.106 # product of RC=0.120 of bitsub-pipe interface multiplied by TC=0.880 for wavelet coming down through pipe-bitsub
DELAY_TANGENTIAL_M1A = 12.548 # in ms    two-way in pipe + top-sub
DELAY_TANGENTIAL_M1B = 13.415 # in ms    two-way in pipe plus two-way in bitsub
DELAY_TANGENTIAL_M1C = 14.282 # in ms    two-way in pipe plus twice two-way in bitsub



AXIAL_FEATURE_EXTRACTOR = dict(
    left_window=  (-.010, -.003), 
    center_window=(-.003, 0.006), 
    right_window= (0.006, 0.018),
    center_pick='max',
    left_pick='min',
    right_pick='min',
)

TANGENTIAL_FEATURE_EXTRACTOR = dict(
    left_window=  (-.01, -.001), 
    center_window=(-.004, 0.004), 
    right_window= (0.002, 0.015),
    center_pick='max',
    left_pick='min',
    right_pick='min',
)

PIPE_ALPHA = 5100 # velocity of the pipe
PIPE_BETA = 2668
PIPE_RHO = 7300 # density of the pipe

CONTACT_FACTOR_AXIAL=1
CONTACT_FACTOR_TANGENTIAL=1

In [ ]:
mod_range = (np.product(list(product((ALPHA_OR_BETA_RANGE/1000)**2, (RHO_RANGE/1000))), axis=1))
min_mod, max_mod = mod_range.min(), mod_range.max()

if COMPONENT == 'axial':
    pipe = Pipe(Rb=Rb, alpha=PIPE_ALPHA, rho=PIPE_RHO, beta=PIPE_BETA, contact_factor=CONTACT_FACTOR_AXIAL)
else:
    pipe = Pipe(Rb=Rb, alpha=PIPE_ALPHA, rho=PIPE_RHO, beta=PIPE_BETA, contact_factor=CONTACT_FACTOR_TANGENTIAL)
    
ax = plt.axes()
for a, r in tqdm(product(ALPHA_OR_BETA_RANGE, RHO_RANGE), total=len(list(product(ALPHA_OR_BETA_RANGE, RHO_RANGE)))):
    mod = ((r/1000)*((a/1000)**2))
    rock = Rock(alpha=a, beta=a, rho=r)

    theoretical = TheoreticalWavelet(pipe, rock, component=COMPONENT, nyquist=5000, filterby=BANDPASS_FILTER)
    time = theoretical.get_time_range_for_window(WINDOW)/1000
##
    raw_primary = theoretical.primary_in_time_domain()
    raw_reflected = theoretical.reflected_in_time_domain()
##    
    if COMPONENT == 'axial':
        pegleg = (theoretical.pegleg_rocksteel(delay_in_ms=DELAY_AXIAL_BITSUB, RC=RC_AXIAL_BITSUB, window=None)) 
        p0     = raw_primary + pegleg

#    
#  This creates first component of Multiple1 ( p0 Primary goes from Rhino down to pipe-bitsub, reflects back to Rhino)
#  this function scales p0 by RC and shifts by DELAY
        m1a = theoretical.apply_time_shift(p0 * RC_AXIAL_M1A, delay_in_ms=DELAY_AXIAL_M1A)          
#            
#  This creates second component of Multiple1 ( p0 Primary goes from Rhino down to bit-rock, reflects back to Rhino)
        m1b_wavelet = signal.convolve(p0, raw_reflected, mode="same")
        m1b_full = theoretical.apply_time_shift(m1b_wavelet * RC_AXIAL_M1B, delay_in_ms=DELAY_AXIAL_M1B)          
#
#  This creates third component of Multiple1
#    function simply applies Theoretical rock/bit convolution, then time shifts for delay including two-way bit-sub transit
#    and RC=0.261 for coefficient off bottom of bit-sub/pipe
        m1c_wavelet = signal.convolve(m1b_wavelet, raw_reflected, mode="same")
        m1c_full = theoretical.apply_time_shift(m1c_wavelet * RC_AXIAL_M1C, delay_in_ms=DELAY_AXIAL_M1C)          
#
#  You can select the wavelet to plot below.  Does not affect anything else. 
#  Must activate only ONE, and must be from this set IF AXIAL

#        wave = raw_primary
#        wave = raw_reflected * -1.0
#        wave = pegleg
#        wave = p0
#        wave = m1b_wavelet
#        wave = m1c_wavelet
#        wave = m1a * RC_AXIAL_TOPSUB
#        wave = m1b_full * RC_AXIAL_TOPSUB
#        wave = m1c_full * RC_AXIAL_TOPSUB
#        wave = (m1a + m1b_full + m1c_full) * RC_AXIAL_TOPSUB
        wave = (p0 + (m1a + m1b_full + m1c_full) * RC_AXIAL_TOPSUB)
        
    else:
#
# TANGENTIAL -- Applying derivative to tangential primary per Jamie (Slack direct message Nov 4, 9:00 am)
#
        raw_primary = theoretical.apply_derivative(raw_primary)
        pegleg = (theoretical.pegleg_rocksteel(delay_in_ms=DELAY_TANGENTIAL_BITSUB, RC=RC_TANGENTIAL_BITSUB, window=None)) 
        pegleg = theoretical.apply_derivative(pegleg)        
        p0 = pegleg + raw_primary
#  
#  This creates first component of Multiple1 (p0 contiues up to CushionSub, reflects down to pipe-bitsub, reflects back up to Rhino)
#  this function scales W by RC and shifts by DELAY
        m1a = theoretical.apply_time_shift(p0 * RC_TANGENTIAL_M1A, delay_in_ms=DELAY_TANGENTIAL_M1A)          
#        
#  This creates second component of Multiple1 ( p0 Primary goes from Rhino down to bit-rock, reflects back to Rhino)
        m1b_wavelet = signal.convolve(p0, raw_reflected, mode="same")
        m1b_full = theoretical.apply_time_shift(m1b_wavelet * RC_TANGENTIAL_M1B, delay_in_ms=DELAY_TANGENTIAL_M1B)          
#
#  This creates third component of Multiple1
#    function simply applies Theoretical rock/bit convolution, then time shifts for delay including two-way bit-sub transit
#    and RC=0.nnn for coefficient off bottom of bit-sub/pipe
        m1c_wavelet = signal.convolve(m1b_wavelet, raw_reflected, mode="same")
        m1c_full = theoretical.apply_time_shift(m1c_wavelet * RC_TANGENTIAL_M1C, delay_in_ms=DELAY_TANGENTIAL_M1C)          
#
#
#  You can select the wavelet to plot below.  Does not affect anything else. 
#  Must activate only ONE, and must be from this set IF TANGENTIAL

#
#        wave = raw_primary
#        wave = raw_reflected
#        wave = pegleg
#        wave = p0
#        wave = m1b_wavelet
#        wave = m1c_wavelet
#        wave = m1a * RC_TANGENTIAL_TOPSUB
#        wave = m1b_full * RC_TANGENTIAL_TOPSUB
#        wave = m1c_full * RC_TANGENTIAL_TOPSUB
#        wave = (m1a + m1b_full + m1c_full) * RC_TANGENTIAL_TOPSUB
#        wave = (p0 + (m1a + m1b_full + m1c_full) * RC_TANGENTIAL_TOPSUB)
#

    w = signal.filtfilt(theoretical.fir_taps, 1, wave)                                                                                                                                      
    wavelet_full = theoretical.get_window_from_center(WINDOW, w)
    
    if COMPONENT == 'axial':
        extracted_features = feature_extractor(wavelet_full, time, **AXIAL_FEATURE_EXTRACTOR)
    else:
        extracted_features = feature_extractor(wavelet_full, time, **TANGENTIAL_FEATURE_EXTRACTOR)
    
    ax.plot(time, wavelet_full, color=cmap((mod-min_mod)/max_mod), zorder=5, label='rho: {:.2f}, velocity: {:.2f}, modulus: {:.2f}'.format(r, a, mod))
    ax.scatter(extracted_features['center']['delay'], extracted_features['center']['amplitude'], s=20, c='k', zorder=10)
    ax.scatter(extracted_features['left']['delay'], extracted_features['left']['amplitude'], s=20, c='k', zorder=10, marker="<")
    ax.scatter(extracted_features['right']['delay'], extracted_features['right']['amplitude'], s=20, c='k', zorder=10, marker=">")

ax.figure.dpi = 100
ax.figure.set_size_inches(20,10)
ax.legend(loc='left', frameon=True)

if COMPONENT == 'axial':
    ax.vlines(AXIAL_FEATURE_EXTRACTOR['center_window'], w.min(), w.max())
    ax.vlines(AXIAL_FEATURE_EXTRACTOR['left_window'], w.min(), w.max())
    ax.vlines(AXIAL_FEATURE_EXTRACTOR['right_window'], w.min(), w.max())
    ax.set_title('component: {}  -  bandpass: {}'.format(COMPONENT, BANDPASS_FILTER, 3))
else:
    ax.vlines(TANGENTIAL_FEATURE_EXTRACTOR['center_window'], w.min(), w.max())
    ax.vlines(TANGENTIAL_FEATURE_EXTRACTOR['left_window'], w.min(), w.max())
    ax.vlines(TANGENTIAL_FEATURE_EXTRACTOR['right_window'], w.min(), w.max())
    ax.set_title('component: {}  -  bandpass: {}'.format(COMPONENT, BANDPASS_FILTER, 3))
    
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.002))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    
t = ax.set_xlabel('Time (s)')

# PARAMETERS FOR THE MODELING

###  Note parameters "RC_xxx_Connector" and "DELAY_xxx_Connector" related to pipe-to-pipe connection that generates another pegleg multiple.
###  This pegleg is introduced in model with this call in the Modeling cell (next after the cell below):


In [ ]:
MINE = 'South Walker Creek'

PIPE_ALPHA = 5100   # velocity of the pipe
PIPE_BETA = 2668
PIPE_RHO = 7300     # density of the pipe

ROCK_ALPHA_RANGE = (500, 4000, 50)        # range of the c velocity modeling
ROCK_BETA_RANGE = (300, 2000, 25)          # range of the s velocity modeling
ROCK_RHO_RANGE = (1500, 3000 + 500, 500)   # range of the density modeling

CONTACT_FACTOR_AXIAL = 1
CONTACT_FACTOR_TANGENTIAL = 1

# Modeling Axial Times  -  fundamental and bitsub pegleg only here

In [ ]:
alpha_range = np.arange(*ROCK_ALPHA_RANGE)
RHO_RANGE = np.arange(*ROCK_RHO_RANGE)

#   Calculate multipass delays for number of pipes
DELAY_AXIAL_M1A = DELAY_AXIAL_M1A + (NUM_PIPES - 1) * DELAY_AXIAL_PIPE
DELAY_AXIAL_M1B = DELAY_AXIAL_M1B + (NUM_PIPES - 1) * DELAY_AXIAL_PIPE
DELAY_AXIAL_M1C = DELAY_AXIAL_M1C + (NUM_PIPES - 1) * DELAY_AXIAL_PIPE
#
window = 310
axial_delays = []
axial_modulus = []
axial_right_to_left_ratio = []
axial_center_to_left_ratio = []

pipe = Pipe(Rb=Rb, alpha=PIPE_ALPHA, rho=PIPE_RHO, beta=PIPE_BETA, contact_factor=CONTACT_FACTOR_AXIAL)
    
axial_wavelets = []
for a, r in tqdm(product(alpha_range, RHO_RANGE), total=len(list(product(alpha_range, RHO_RANGE)))):
    mod = ((r/1000)*((a/1000)**2))
    rock = Rock(alpha=a, beta=a, rho=r)

    theoretical = TheoreticalWavelet(pipe, rock, component=COMPONENT, nyquist=5000, filterby=BANDPASS_FILTER)
    time = theoretical.get_time_range_for_window(WINDOW)/1000
##
    raw_primary = theoretical.primary_in_time_domain()
    raw_reflected = theoretical.reflected_in_time_domain()    
#
    pegleg = (theoretical.pegleg_rocksteel(delay_in_ms=DELAY_AXIAL_BITSUB, RC=RC_AXIAL_BITSUB, window=None)) 
    p0     = raw_primary + pegleg
#    
# The call below generates a pegleg for pipe-to-pipe connection. NUM_PIPE is number of steels in drillstring. 
#
    count = NUM_PIPES
    while (count > 1):
        p0 += theoretical.apply_time_shift(p0 * RC_AXIAL_Connector, delay_in_ms=DELAY_AXIAL_Connector)
        count = count - 1
#
#  This creates first component of Multiple1 ( p0 Primary goes from Rhino down to pipe-bitsub, reflects back to Rhino)
#  this function scales p0 by RC and shifts by DELAY
    m1a = theoretical.apply_time_shift(p0 * RC_AXIAL_M1A, delay_in_ms=DELAY_AXIAL_M1A)          
#            
#  This creates second component of Multiple1 ( p0 Primary goes from Rhino down to bit-rock, reflects back to Rhino)
    m1b_wavelet = signal.convolve(p0, raw_reflected, mode="same")
    m1b_full = theoretical.apply_time_shift(m1b_wavelet * RC_AXIAL_M1B, delay_in_ms=DELAY_AXIAL_M1B)          
#
#  This creates third component of Multiple1
#    function simply applies Theoretical rock/bit convolution, then time shifts for delay including two-way bit-sub transit
#    and RC=0.261 for coefficient off bottom of bit-sub/pipe
    m1c_wavelet = signal.convolve(m1b_wavelet, raw_reflected, mode="same")
    m1c_full = theoretical.apply_time_shift(m1c_wavelet * RC_AXIAL_M1C, delay_in_ms=DELAY_AXIAL_M1C)          
#

#  You can select the wavelet to plot below.  Does not affect anything else.
#    wave = raw_primary
#    wave = raw_reflected
#    wave = pegleg
#    wave = p0
#    wave = m1b_wavelet
#    wave = m1c_wavelet
#    wave = m1a * RC_AXIAL_TOPSUB
#    wave = m1b_full * RC_AXIAL_TOPSUB
#    wave = m1c_full * RC_AXIAL_TOPSUB
#    wave = (m1a + m1b_full + m1c_full) * RC_AXIAL_TOPSUB
    wave = (p0 + (m1a + m1b_full + m1c_full) * RC_AXIAL_TOPSUB)

    
    axial_to_pipeline = wave.copy()
    w = signal.filtfilt(theoretical.fir_taps, 1, wave)                                                                                                                                      
    w = theoretical.get_window_from_center(WINDOW, w)
        
    extracted_features = feature_extractor(w, time, **AXIAL_FEATURE_EXTRACTOR)
    delay = extracted_features['center']['delay']
    amplitude = extracted_features['center']['amplitude']
    right_to_left_ratio = extracted_features['right_to_left_ratio']
    center_to_left_ratio = extracted_features['center_to_left_ratio']
    
    
    axial_delays.append(delay)
    axial_right_to_left_ratio.append(right_to_left_ratio)
    axial_center_to_left_ratio.append(center_to_left_ratio)
    
    mod = ((r/1000)*((a/1000)**2))
    axial_modulus.append(mod)
    axial_wavelets.append(axial_to_pipeline)
    
axial_delays = np.asarray(axial_delays)
# axial_delays = axial_delays + (0.0000001 - axial_delays.min())

axial_modulus = np.asarray(axial_modulus)
axial_right_to_left_ratio = np.asarray(axial_right_to_left_ratio)
axial_center_to_left_ratio = np.asarray(axial_center_to_left_ratio)

In [ ]:
axial_wavelets = [i.tolist() for i in axial_wavelets]

## Quick QC Axial

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15+15, 5))
axes[0].plot(time, w)
axes[0].scatter(delay, amplitude, s=50, c='r')
axes[0].scatter(extracted_features['left']['delay'], extracted_features['left']['amplitude'], s=50, c='r', marker="<")
axes[0].scatter(extracted_features['right']['delay'], extracted_features['right']['amplitude'], s=50, c='r', marker=">")
axes[0].set_title('Example Wavelet - alpha: {}, rho: {}'.format(a, r))
axes[1].scatter(axial_delays, axial_modulus)
axes[1].set_xlabel('Picked Times (s)'); axes[1].set_ylabel('CompressionalModulus(GPa)')
axes[1].set_xlim(axial_delays.min(), axial_delays.max())
axes[1].set_title('Time vs Modulus')
axes[2].scatter(axial_right_to_left_ratio, axial_modulus)
axes[2].set_xlabel('RHS/LHS'); axes[2].set_ylabel('CompressionalModulus(GPa)')
axes[2].set_xlim(axial_right_to_left_ratio.min(), axial_right_to_left_ratio.max())
axes[2].set_title('RHS/LHS vs Modulus')
axes[3].scatter(axial_center_to_left_ratio, axial_modulus)
axes[3].set_xlabel('CENTER/LHS'); axes[3].set_ylabel('CompressionalModulus(GPa)')
axes[3].set_xlim(axial_center_to_left_ratio.min(), axial_center_to_left_ratio.max())
axes[3].set_title('CENTER/LHS vs Modulus')

# Modeling Tangential Times  -  fundamental and bitsub pegleg only here

In [ ]:
beta_range = np.arange(*ROCK_BETA_RANGE)
RHO_RANGE = np.arange(*ROCK_RHO_RANGE)

#   Calculate multipass delays for number of pipes
DELAY_TANGENTIAL_M1A = DELAY_TANGENTIAL_M1A + (NUM_PIPES - 1) * DELAY_TANGENTIAL_PIPE
DELAY_TANGENTIAL_M1B = DELAY_TANGENTIAL_M1B + (NUM_PIPES - 1) * DELAY_TANGENTIAL_PIPE
DELAY_TANGENTIAL_M1C = DELAY_TANGENTIAL_M1C + (NUM_PIPES - 1) * DELAY_TANGENTIAL_PIPE
        
window = 310
tangential_delays = []
tangential_modulus = []
tangential_right_to_left_ratio = []
tangential_center_to_left_ratio = []


pipe = Pipe(Rb=Rb, alpha=PIPE_ALPHA, rho=PIPE_RHO, beta=PIPE_BETA, contact_factor=CONTACT_FACTOR_TANGENTIAL)

wavelets = []
for b, r in tqdm(product(beta_range, RHO_RANGE), total=len(list(product(beta_range, RHO_RANGE)))):
    rock = Rock(beta=b, rho=r, component='tangential')
    theoretical = TheoreticalWavelet(pipe, rock, component='tangential', nyquist=5000, filterby=BANDPASS_FILTER)
    time = theoretical.get_time_range_for_window(WINDOW)/1000
#
    raw_primary = theoretical.primary_in_time_domain()
    raw_reflected = theoretical.reflected_in_time_domain()
    raw_primary = theoretical.apply_derivative(raw_primary)
    pegleg = (theoretical.pegleg_rocksteel(delay_in_ms=DELAY_TANGENTIAL_BITSUB, RC=RC_TANGENTIAL_BITSUB, window=None)) 
    pegleg = theoretical.apply_derivative(pegleg)        
    p0 = pegleg + raw_primary
#

# The call below generates a pegleg for pipe-to-pipe connection. NUM_PIPE is number of steels in drillstring. 
#
    count = NUM_PIPES
    while (count > 1):
        p0 += theoretical.apply_time_shift( p0 * RC_TANGENTIAL_Connector, delay_in_ms=DELAY_TANGENTIAL_Connector)
        count = count - 1
#
    m1a = theoretical.apply_time_shift(p0 * RC_TANGENTIAL_M1A, delay_in_ms=DELAY_TANGENTIAL_M1A)                  
    m1b_wavelet = signal.convolve(p0, raw_reflected, mode="same")
    m1b_full = theoretical.apply_time_shift(m1b_wavelet * RC_TANGENTIAL_M1B, delay_in_ms=DELAY_TANGENTIAL_M1B)          
    m1c_wavelet = signal.convolve(m1b_wavelet, raw_reflected, mode="same")
    m1c_full = theoretical.apply_time_shift(m1c_wavelet * RC_TANGENTIAL_M1C, delay_in_ms=DELAY_TANGENTIAL_M1C)          
#    
#
#  You can select the wavelet to plot below.  Does not affect anything else.
#    wave = raw_primary
#    wave = raw_reflected
#    wave = pegleg
#    wave = p0
#    wave = m1b_wavelet
#    wave = m1c_wavelet
#    wave = m1a * RC_TANGENTIAL_TOPSUB
#    wave = m1b_full * RC_TANGENTIAL_TOPSUB
#    wave = m1c_full * RC_TANGENTIAL_TOPSUB
#    wave = (m1a + m1b_full + m1c_full) * RC_TANGENTIAL_TOPSUB
    wave = (p0 + (m1a + m1b_full + m1c_full) * RC_TANGENTIAL_TOPSUB)
#

    w = signal.filtfilt(theoretical.fir_taps, 1, wave)                                                                                                                                      
    w = theoretical.get_window_from_center(WINDOW, w)
        
    extracted_features = feature_extractor(w, time, **TANGENTIAL_FEATURE_EXTRACTOR)
    delay = extracted_features['center']['delay']
    amplitude = extracted_features['center']['amplitude']
    right_to_left_ratio = extracted_features['right_to_left_ratio']
    center_to_left_ratio = extracted_features['center_to_left_ratio']
    
    
    tangential_delays.append(delay)
    tangential_right_to_left_ratio.append(right_to_left_ratio)
    tangential_center_to_left_ratio.append(center_to_left_ratio)
    
    mod = ((r/1000)*((b/1000)**2))
    tangential_modulus.append(mod)
    
tangential_delays = np.asarray(tangential_delays)

tangential_modulus = np.asarray(tangential_modulus)
tangential_right_to_left_ratio = np.asarray(tangential_right_to_left_ratio)
tangential_center_to_left_ratio = np.asarray(tangential_center_to_left_ratio)

## Quick QC Tangential

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15+15, 5))
axes[0].plot(time, w)
axes[0].scatter(delay, amplitude, s=50, c='r')
axes[0].scatter(extracted_features['left']['delay'], extracted_features['left']['amplitude'], s=50, c='r', marker="<")
axes[0].scatter(extracted_features['right']['delay'], extracted_features['right']['amplitude'], s=50, c='r', marker=">")
axes[0].set_title('Example Wavelet - beta: {}, rho: {}'.format(a, r))
axes[1].scatter(tangential_delays, tangential_modulus)
axes[1].set_xlabel('Picked Times (s)'); axes[1].set_ylabel('ShearModulus(GPa)')
axes[1].set_xlim(tangential_delays.min(), tangential_delays.max())
axes[1].set_title('Time vs Modulus')
axes[2].scatter(tangential_right_to_left_ratio, tangential_modulus)
axes[2].set_xlabel('RHS/LHS'); axes[2].set_ylabel('ShearModulus(GPa)')
axes[2].set_xlim(tangential_right_to_left_ratio.min(), tangential_right_to_left_ratio.max())
axes[2].set_title('RHS/LHS vs Modulus')
axes[3].scatter(tangential_center_to_left_ratio, tangential_modulus)
axes[3].set_xlabel('CENTER/LHS'); axes[3].set_ylabel('ShearModulus(GPa)')
axes[3].set_xlim(tangential_center_to_left_ratio.min(), tangential_center_to_left_ratio.max())
axes[3].set_title('CENTER/LHS vs Modulus')

# Check Distribution of Axial Delays and Tangential Delays 

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15,5))

sns.distplot(axial_delays, label='axial_time', ax=ax1, kde=False, bins=35)
sns.distplot(tangential_delays, label='tangential_time', ax=ax2, kde=False, bins=35)

## A bigger plot of the picked times.  You may need to change range of axes[n].set.xlim at bottom of cell.

In [ ]:
import matplotlib.ticker as ticker

fig, axes = plt.subplots(1, 2, figsize=(25,15), dpi=200, sharey=False)
sns.scatterplot(axial_delays, axial_modulus, ax=axes[0])
sns.scatterplot(tangential_delays, tangential_modulus, ax=axes[1])

#    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
#    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.00025))
#    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())

axes[0].set_xlabel('primary time')
axes[0].set_ylabel('Compressional Modulus (GPa)')

axes[1].set_xlabel('primary time')
axes[1].set_ylabel('Shear Modulus (GPa)')

axes[0].set_xlim(-0.0010, 0.008)
axes[1].set_xlim(-0.0035, -0.0005)

Modulus is a function of time.

# Modeling

## This is a bunch of functions we can fit our model to. 

In [ ]:
from theory.function_handler import ModelingFunction

In [ ]:
line_function = ModelingFunction('a * x + b')

power_func = ModelingFunction('a * (x + b)**(c)')
power_func2 = ModelingFunction('(((a * (((x)+b)**c) + d) + (e*x) + f) / (g  * ((x) ** (2)) + h * x + i) * j)')
power_func3 = ModelingFunction('((a * (((x)+b)**c) + d) / (e  * ((x) ** (2)) + f * x + g) * h)')
power_func4 = ModelingFunction('((a * (((x)+b)**c) + d)  / (e * (x) + f) * g)')
exponential_func = ModelingFunction('a * np.exp(-b * np.asarray(x)) + c')
quadratic_func = ModelingFunction('(a * x**2) + (b * x) + c')
cubic_func = ModelingFunction('(a * x**3) + (b * x**2) + (c * x) + d')
quartic_func = ModelingFunction('(a * x**4) + (b * x**3) + (c * x**2) + (d * x) + e')

## SELECT WHICH FUNCTION TO USE.  Work in conjunction with range of moduli to fit (next active cell)

In [ ]:
# SELECT FUNCTION TO USE

from copy import copy 
#current_axial_func = copy(quadratic_func)
#current_axial_func = copy(quartic_func)
current_axial_func = copy(power_func4)
current_tangential_func = copy(power_func3)

## Removing modulus for the fitting  --  MUST CHECK FOR APPROPRIATE RANGE AND ENTER BELOW

In [ ]:
axial_delays_clipped = axial_delays[((axial_modulus) < 10) & (axial_modulus > 0.9)]
axial_modulus_clipped = axial_modulus[((axial_modulus) < 10) & (axial_modulus > 0.9)]
                                      
tangential_delays_clipped = tangential_delays[((tangential_modulus) < 12) & (tangential_modulus > 1.5)]
tangential_modulus_clipped = tangential_modulus[((tangential_modulus) < 12) & (tangential_modulus > 1.5)]

In [ ]:
current_axial_func.fit( 
    axial_delays_clipped,
    axial_modulus_clipped)

current_tangential_func.fit( 
    tangential_delays_clipped,
    tangential_modulus_clipped,)

## Look at the equations 

In [ ]:
print('axial: ', current_axial_func.get_fitted_string())
print()
print('tangential: ', current_tangential_func.get_fitted_string())

In [ ]:
import matplotlib.ticker as ticker
import matplotlib

In [ ]:
matplotlib.use('agg')

## Plot curves and evaluate the fit.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(25,15), dpi=200, sharey=True)
sns.scatterplot(axial_delays_clipped, axial_modulus_clipped, ax=axes[0])
sns.scatterplot(tangential_delays_clipped, tangential_modulus_clipped, ax=axes[1])
for ax in axes.ravel():
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.0005))
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    
axes[0].plot(np.linspace(axial_delays_clipped.min(), axial_delays_clipped.max()), current_axial_func(np.linspace(axial_delays_clipped.min(), axial_delays_clipped.max())), '-', color='r')
axes[1].plot(np.linspace(tangential_delays_clipped.min(), tangential_delays_clipped.max()), current_tangential_func(np.linspace(tangential_delays_clipped.min(), tangential_delays_clipped.max())), '-', color='r')


axes[0].set_xlabel('Primary Time')
axes[0].set_ylabel('CompressionalModulus(GPa)')
axes[1].set_xlabel('Primary Time')
axes[1].set_ylabel('ShearModulus(GPa)')

axes[0].set_xlim(-0.0008, axial_delays.max())
axes[1].set_xlim(-0.0035, tangential_delays.max())

# for ax in axes.ravel():
#    ax.set_ylim(0, MODULUS_RANGE[1])
#     ax.set_yscale('log')

axes[0].set_xlim(-0.0008, 0.008)
# axes[1].set_xlim(-.002, -.0015)
fig.suptitle(MINE)

## Need to carefully evaluate curve fit.  May take multiple attempts changing min/max moduli and selected function.

## Output the functions   -   Bandpass is automatic.
### -   Identify Mine/Project, Drill number, and NUM_PIPES.   
### -   Would be good to have automatic substitution for output file naming.

In [ ]:
bandpass_str = '-'.join([str(s) for s in BANDPASS_FILTER])

current_axial_func.save('SWC_Axial_Primary+Mult1+Drill16+Pipes6_{}.model'.format(bandpass_str))
current_tangential_func.save('SWC_Tangential_Primary+Mult1+Drill16+Pipes6_{}.model'.format(bandpass_str))

In [ ]:
axial_lookup_table = pd.DataFrame(np.c_[np.asarray(list(product(alpha_range, RHO_RANGE))), axial_modulus, axial_delays], columns=['alpha', 'rho', 'modulus', 'primary_time'])
axial_lookup_table.to_csv('SWC_Axial_Primary+Mult1+Drill16+Pipes6_{}_lookup.csv'.format(bandpass_str), index=False)

tangential_lookup_table = pd.DataFrame(np.c_[np.asarray(list(product(beta_range, RHO_RANGE))), tangential_modulus, tangential_delays], columns=['beta', 'rho', 'modulus', 'primary_time'])
tangential_lookup_table.to_csv('SWC_Tangential_Primary+Mult1+Drill16+Pipes6_{}_lookup.csv'.format(bandpass_str), index=False)

# Read the Lookup Table back in and plot it to verify

In [ ]:
from scipy.interpolate import interp1d

def lookup_table(shifted_time, modulus):
    return interp1d(shifted_time, modulus, fill_value="extrapolate")

In [ ]:
axial_lookup = pd.read_csv('SWC_Axial_Primary+Mult1+Drill16+Pipes6_{}_lookup.csv'.format(bandpass_str))
tangential_lookup = pd.read_csv('SWC_Tangential_Primary+Mult1+Drill16+Pipes6_{}_lookup.csv'.format(bandpass_str))

ALookupTable = lookup_table(axial_lookup.primary_time, axial_lookup.modulus)
TLookupTable = lookup_table(tangential_lookup.primary_time, tangential_lookup.modulus)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(25,15), dpi=200, sharey=True)
sns.scatterplot(axial_delays, axial_modulus, ax=axes[0])
sns.scatterplot(tangential_delays, tangential_modulus, ax=axes[1])

axes[0].plot(np.linspace(axial_delays.min(), axial_delays.max()), ALookupTable(np.linspace(axial_delays.min(), axial_delays.max()).reshape(-1, 1)), '-', color='r')
axes[1].plot(np.linspace(tangential_delays.min(), tangential_delays.max()), TLookupTable(np.linspace(tangential_delays.min(), tangential_delays.max()).reshape(-1, 1)), '-', color='r')

axes[0].set_xlabel('Primary Time')
axes[0].set_ylabel('CompressionalModulus(GPa)')
axes[1].set_xlabel('Primary Time')
axes[1].set_ylabel('ShearModulus(GPa)')